In [ ]:
                    # SUFORMUOJAME all_beats_attr

for SubjCode in SubjCodes:
    # print("\nECG įrašas:", SubjCode)
    
    # Nuskaitome paciento anotacijas ir jų indeksus
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)

    # Ciklas per visas paciento įrašo anotacijas (simbolius) ir jų vietas (i_sample)
    for i, i_sample in enumerate(atr_sample):
    
        beats_attr = {'userNr':int(userNr), 'recordingNr':int(recordingNr), 'sample':int(i_sample), 'symbol':str(atr_symbol[i])}
        df_new_row = pd.DataFrame([beats_attr])
        all_beats_attr = pd.concat([all_beats_attr, df_new_row])

# Ciklo per pacientų įrašus pabaiga




                    # SUFORMUOJAME validation_set_stats

validation_set_stats = pd.DataFrame(columns=['idx', 'test_label', 'pred_label', 'SubjCode'])

# Ciklas per pacientų įrašus
for SubjCode in SubjCodes:

# ------ I. Nuskaitome įrašo atributus ---------- 

    signal = read_rec(rec_dir, SubjCode)
    userId, recId, file_name = get_recId(rec_dir, userNr, recNr)
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)

# ------ II. Suformuojame testinius ir priskirtus pūpsnių klasių numerius ---------- 

    test_labels = np.array([all_beats[symbol] for symbol in atr_symbol])
    pred_labels = predict_cnn_fda_vu_v1_micro(signal, atr_sample, model_dir)

    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recNr)]
    index_start = selected_ind[0]

# ------ III. Suformuojame validatio_st_stats, test_y ir pred_y ---------- 

    for idx in range(len(atr_sample)):
        index_current = index_start + idx
        validation_set_stats = validation_set_stats.append({'idx':index_current, 
        'test_label':test_labels[idx],'pred_label':pred_labels[idx], 'SubjCode': SubjCode}, ignore_index=True)

    test_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['test_label']).astype('int') 
    pred_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['pred_label']).astype('int')
   
# ------ IV. apskaičiuojame tikslumą  ---------- 

    confusion = confusion_matrix(test_y_mod, pred_y_mod)
    prec,rec,fsc,sup = precision_recall_fscore_support(test_y_mod, pred_y_mod, labels=[0, 1, 2], zero_division=0)

# Ciklo per pacientų įrašus pabaiga

# Suformuojame :
validate_ind_lst = list(validation_set_stats['idx'])
y_validate = np.array(validation_set_stats['test_label']).astype('int')
y_predicted = np.array(validation_set_stats['pred_label']).astype('int')

# Tikslas: atsikratyti validation_set_stats, viską gauti, ko reikia iš all_beats_attr